In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append('../')

In [ ]:
from src.d02_intermediate.classifier_data_api import ClassifierDataApi
from src.d04_modeling import propositional_classifier as pc
from src.d04_modeling.naive_classifier import NaiveClassifier

We load the data with an arbitrary classifier, and get an instance of the ClassifierDataApi class to access the data:

In [ ]:
load_data = pc.andClassifier([])
cda = ClassifierDataApi()

# Eligibility Criteria:

This work is an extension of the Neighborhood Proxy Criteria. We would like to replace that for a more general eligibitliy criteria that takes into account:

1. Public housing projects in a block (data provided by the School District, SFHA_ex_Sr column)
2. Red-lining historical maps

Blocks satisfying at least one of these criteria are eligible to receive the equity tiebreaker.

We also address transparency issues. The eligibility criteria never automatically give a block the equity tiebreaker, so that knowing these criteria is not enough for a family to "game" the system by moving. After blocks are pre-selected by these criteria, the demographic composition of the block should be taken into account to actually decide upon the tiebreaker.

More criteria can be incorporated, and we suggest that criteria which measure the development status and number of resources in a neighborhood are incorporated here. For example, accessibility and public transportation or gentrification. Importantly, the eligibility criteria can be provided at a coarser level than the blocks, so that more datasets are potentially available.

## 1. San Francisco Public Housing

The first criteria qualifying a block for the equity tiebreaker is whether there are non-senior public housing developments in that block. We can visualize these blocks in the map below:

In [ ]:
housing_gdf = cda.get_map_df_data("Housing")

fig, ax = plt.subplots(figsize=(30,30))
ax = housing_gdf.plot(column="Housing", ax=ax)
ax.set_title("Blocks with non-Senior SF Housing", fontsize=50)
plt.show()

This a very limited number of blocks (65). Other proxies for affordable housing could be considered, such as the SA Housing rathern than SFHA Housing. If we award a tiebreaker to everyone with public housing:

In [ ]:
c = pc.andClassifier(["Housing"], binary_var=[0])

print("{:.1%} of all the students receive the tiebreaker\n".format(c.pr([])))

print("{:.1%} of all the focal students receive a tiebreaker".format(c.tpr([])))
print("{:.1%} of all the non-focal students receive a tiebreaker".format(c.fpr([])))
print("{:.1%} of the students receiving a tiebreaker are non-focal".format(1-c.precision([])))

## 2. Red-Lining History

Another dataset we would like to incorporate is historical red-lining maps. This data is available publicaly and is immutable: https://dsl.richmond.edu/panorama/redlining/#loc=12/37.83/-122.464&city=san-francisco-ca&area=null&text=downloads